## 1. **Configure the Notebook:** 

In [1]:
# Import necessary libraries
import numpy as np
from xarray_healpy import HealpyGridInfo, HealpyRegridder
from pangeo_fish.grid import center_longitude
import xarray as xr
import pandas as pd
import xdggs
from pangeo_fish.hmm.estimator import EagerEstimator
from pangeo_fish.hmm.optimize import EagerBoundsSearch
from pangeo_fish.utils import temporal_resolution
from pangeo_fish.hmm.prediction import Gaussian1DHealpix,Gaussian2DCartesian
from tlz.functoolz import curry
from pint_xarray import unit_registry as ureg

# earth_radius defines the radius of the Earth used for distance calculations.
earth_radius = ureg.Quantity(6371, "km")
# maximum_speed sets the maximum allowable speed for the tagged fish.
maximum_speed = ureg.Quantity(60, "km / day")

# It will factor the allowed maximum displacement of the fish.
adjustment_factor = 5
# truncate sets the truncating factor for computed maximum allowed sigma for convolution process.
truncate = 4
tolerance = 1e-6
# Defines default chunk size for optimisation.
chunk_time = 24
default_chunk = {"time": chunk_time, "lat": -1, "lon": -1}
default_chunk_xy = {"time": chunk_time, "x": -1, "y": -1}
default_chunk_cells = {"time": chunk_time, "cells": -1}

/home/ecap/micromamba/envs/pangeo-fish/lib/python3.11/site-packages/movingpandas/__init__.py:37: UserWarning: Missing optional dependencies. To use the trajectory smoother classes please install Stone Soup (see https://stonesoup.readthedocs.io/en/latest/#installation).
  warnings.warn(e.msg, UserWarning)


In [2]:
import xarray as xr
emission = xr.open_dataset('../A19124/combined.zarr',
                           engine="zarr",
                           chunks={},
                           inline_array=True,
                           storage_options=None,
                        )
emission

<xarray.Dataset> Size: 409MB
Dimensions:     (y: 294, x: 659, time: 258)
Coordinates:
    cell_ids    (y, x) int64 2MB dask.array<chunksize=(294, 659), meta=np.ndarray>
    latitude    (y, x) float64 2MB dask.array<chunksize=(294, 659), meta=np.ndarray>
    longitude   (y, x) float64 2MB dask.array<chunksize=(294, 659), meta=np.ndarray>
    resolution  float64 8B ...
  * time        (time) datetime64[ns] 2kB 2022-06-13T12:00:00 ... 2022-06-24T...
Dimensions without coordinates: y, x
Data variables:
    final       (y, x) float64 2MB dask.array<chunksize=(294, 659), meta=np.ndarray>
    initial     (y, x) float64 2MB dask.array<chunksize=(294, 659), meta=np.ndarray>
    mask        (y, x) float64 2MB dask.array<chunksize=(294, 659), meta=np.ndarray>
    pdf         (time, x, y) float64 400MB dask.array<chunksize=(24, 659, 294), meta=np.ndarray>
Attributes:
    grid_type:  healpix
    lat:        0
    level:      12
    lon:        0
    nside:      4096
    rot_lat:    0
    rot_lon:    0

In [3]:
def sigma_to_radian(sigma, earth_radius_km=6371):
    """
    Convert sigma (arc length) to radians.
    
    Parameters:
    - sigma: float, the arc length in the same unit as earth_radius_km.
    - earth_radius_km: float, the Earth's radius in kilometers (default: 6371 km).
    
    Returns:
    - float: sigma in radians.
    """
    return np.sqrt(4*np.pi/(12*4096**2))*sigma

# Apply the function to sigma = 1.06
sigma = 1.06292
sigma_radian = sigma_to_radian(sigma)

print(f"Sigma in radians: {sigma_radian}")
earth_radius_ = xr.DataArray(earth_radius, dims=None)

timedelta = temporal_resolution(emission["time"]).pint.quantify().pint.to("h")
grid_resolution = earth_radius_ * emission["resolution"].pint.quantify()

maximum_speed_ = xr.DataArray(maximum_speed, dims=None).pint.to("km / h")
max_grid_displacement = maximum_speed_ * timedelta * adjustment_factor / grid_resolution
max_sigma = max_grid_displacement.pint.to("dimensionless").pint.magnitude / truncate

max_grid_displacement_radian = maximum_speed_ * timedelta * adjustment_factor / earth_radius_
max_sigma_radian = max_grid_displacement_radian.pint.to("dimensionless").pint.magnitude / truncate

# Applying max sigma over the emission matrices
emission.attrs["max_sigma"] = max_sigma_radian.item()
print(max_sigma.item())
print(max_sigma_radian)

Sigma in radians: 0.00026555527939709026
1.963306279189256
0.0004905038455501491


/home/ecap/pangeo-fish/pangeo_fish/utils.py:122: FutureWarning: Hour.delta is deprecated and will be removed in a future version. Use pd.Timedelta(obj) instead
  timedelta = to_offset(freq).delta.to_numpy()


In [4]:
# Set required attributes cleanly
emission["cell_ids"].attrs["grid_name"] = "healpix"
emission["cell_ids"].attrs["level"] = 12
#emission_1D["cell_ids"].attrs["nest"] = True
# Récupérer les attributs existants de cell_ids
attrs_to_keep = ["level", "grid_name"]
# should also have nest
emission["cell_ids"].attrs = {key: value for key, value in emission["cell_ids"].
                              attrs.items() if key in attrs_to_keep}

In [5]:
emission_full_one = emission.copy() 
emission_full_one['pdf'] = xr.ones_like(emission_full_one['pdf'])

In [6]:
emission = emission_full_one.compute()  # WHAT DOES IT CHANGE?
#emission = emission.pipe(xdggs.decode)
predictor_factory = curry(
    Gaussian2DCartesian,
    truncate=4.0
)

In [7]:
##############################
# MANUELLEMENT AJOUT DU SIGMA
#############################
optimized = EagerEstimator(
            sigma= 1.06292,
            predictor_factory=predictor_factory)

optimized

EagerEstimator(predictor_factory=<class 'pangeo_fish.hmm.prediction.Gaussian2DCartesian'>, sigma=1.06292, predictor=None)

In [8]:
emission

<xarray.Dataset> Size: 409MB
Dimensions:     (y: 294, x: 659, time: 258)
Coordinates:
    cell_ids    (y, x) int64 2MB 55996335 56018181 ... 58020672 58020674
    latitude    (y, x) float64 2MB 42.45 42.46 42.47 42.49 ... 54.05 54.06 54.08
    longitude   (y, x) float64 2MB -7.398 -7.4 -7.402 ... -1.148 -1.149 -1.149
    resolution  float64 8B 0.0002498
  * time        (time) datetime64[ns] 2kB 2022-06-13T12:00:00 ... 2022-06-24T...
Dimensions without coordinates: y, x
Data variables:
    final       (y, x) float64 2MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    initial     (y, x) float64 2MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    mask        (y, x) float64 2MB nan nan nan nan nan ... nan nan nan nan nan
    pdf         (time, x, y) float64 400MB 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
Attributes:
    grid_type:  healpix
    lat:        0
    level:      12
    lon:        0
    nside:      4096
    rot_lat:    0
    rot_lon:    0
    max_sigma:  0.0004905038455501491

In [9]:
emission['pdf'] = emission['pdf'].transpose('time', 'y', 'x')

In [10]:
states_1D = optimized.predict_proba(emission)
states_1D = states_1D.to_dataset().chunk(default_chunk_xy).persist()
states_1D

<xarray.Dataset> Size: 405MB
Dimensions:     (y: 294, x: 659, time: 258)
Coordinates:
    cell_ids    (y, x) int64 2MB dask.array<chunksize=(294, 659), meta=np.ndarray>
    latitude    (y, x) float64 2MB dask.array<chunksize=(294, 659), meta=np.ndarray>
    longitude   (y, x) float64 2MB dask.array<chunksize=(294, 659), meta=np.ndarray>
    resolution  float64 8B 0.0002498
  * time        (time) datetime64[ns] 2kB 2022-06-13T12:00:00 ... 2022-06-24T...
Dimensions without coordinates: y, x
Data variables:
    states      (time, y, x) float64 400MB dask.array<chunksize=(24, 294, 659), meta=np.ndarray>

In [11]:
states_1D_from_2D = states_1D.stack(cells=("y", "x")) # change name states_1D_from_2D

In [12]:
states_1D_from_2D

<xarray.Dataset> Size: 409MB
Dimensions:     (cells: 193746, time: 258)
Coordinates:
    cell_ids    (cells) int64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    latitude    (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    longitude   (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    resolution  float64 8B 0.0002498
  * time        (time) datetime64[ns] 2kB 2022-06-13T12:00:00 ... 2022-06-24T...
  * cells       (cells) object 2MB MultiIndex
  * y           (cells) int64 2MB 0 0 0 0 0 0 0 ... 293 293 293 293 293 293 293
  * x           (cells) int64 2MB 0 1 2 3 4 5 6 ... 652 653 654 655 656 657 658
Data variables:
    states      (time, cells) float64 400MB dask.array<chunksize=(24, 193746), meta=np.ndarray>

In [17]:
states_1D_from_2D.pipe(xdggs.decode).isel(time=-1).compute().states.dggs.explore(alpha=0.8)

Map(custom_attribution='', layers=(SolidPolygonLayer(filled=True, get_fill_color=arro3.core.ChunkedArray<Fixed…

In [14]:

states_1D_from_2D#.states.isel(time=2).dropna('cells').cell_ids.compute().data

<xarray.Dataset> Size: 409MB
Dimensions:     (cells: 193746, time: 258)
Coordinates:
    cell_ids    (cells) int64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    latitude    (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    longitude   (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    resolution  float64 8B 0.0002498
  * time        (time) datetime64[ns] 2kB 2022-06-13T12:00:00 ... 2022-06-24T...
  * cells       (cells) object 2MB MultiIndex
  * y           (cells) int64 2MB 0 0 0 0 0 0 0 ... 293 293 293 293 293 293 293
  * x           (cells) int64 2MB 0 1 2 3 4 5 6 ... 652 653 654 655 656 657 658
Data variables:
    states      (time, cells) float64 400MB dask.array<chunksize=(24, 193746), meta=np.ndarray>

In [15]:
states_1D_from_2D = states_1D_from_2D.reset_index('cells')
#states_1D_from_2D = states_1D_from_2D.drop_vars('cells')
states_1D_from_2D.to_zarr(
    f"states_from_emission_2D_emissionfullone.zarr",
    mode="w",
    consolidated=True,
    storage_options=None,
)

In [16]:
states_1D_from_2D

<xarray.Dataset> Size: 408MB
Dimensions:     (cells: 193746, time: 258)
Coordinates:
    cell_ids    (cells) int64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    latitude    (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    longitude   (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    resolution  float64 8B 0.0002498
  * time        (time) datetime64[ns] 2kB 2022-06-13T12:00:00 ... 2022-06-24T...
    y           (cells) int64 2MB 0 0 0 0 0 0 0 ... 293 293 293 293 293 293 293
    x           (cells) int64 2MB 0 1 2 3 4 5 6 ... 652 653 654 655 656 657 658
Dimensions without coordinates: cells
Data variables:
    states      (time, cells) float64 400MB dask.array<chunksize=(24, 193746), meta=np.ndarray>